This code will eventually go into a pypi package

In [ ]:
import pickle
import pandas
import json
from sklearn.mixture import GaussianMixture
import os

def hyperparameter(s):
    j = json.loads(os.environ['SC_MODEL_HYPERPARAMS'])
    return j.get(s)
def model_cfg(s):
    j = json.loads(os.environ['SC_MODEL_CFG'])
    return j.get(s)

def write_fit(fit):
    with open(os.environ['SC_MODEL_FIT_DEST'], 'wb') as handle:
        pickle.dump(fit, handle, protocol=pickle.HIGHEST_PROTOCOL)

def set_metrics(ms):
    with open(os.environ['SC_MODEL_METRICS_DEST'], 'w') as fp:
        json.dump(ms, fp)

def data_file_path():
    return os.environ['SC_MODEL_DATA_FILE']

This is the code to train

In [ ]:
df = pandas.read_csv(data_file_path())
dfxs = df.iloc[:, :-1]
dfy = df.iloc[:, -1]
modeltype = model_cfg('regression_model')

if modeltype == "Linear":
    from sklearn.linear_model import LinearRegression
    model = LinearRegression().fit(dfxs, dfy)
elif modeltype == "Ridge":
    from sklearn.linear_model import Ridge
    model = Ridge(alpha=hyperparameter('regularization')).fit(dfxs, dfy)
elif modeltype == "Lasso":
    from sklearn.linear_model import Lasso
    model = Lasso(alpha=hyperparameter('regularization')).fit(dfxs, dfy)
elif modeltype == "Random Forest":
    from sklearn.ensemble import RandomForestRegressor
    model = RandomForestRegressor().fit(dfxs, dfy)
elif modeltype == "Partial Least Squares":
    from sklearn.cross_decomposition import PLSRegression
    model = PLSRegression(n_components=hyperparameter('components')).fit(dfxs, dfy)
elif modeltype == "Support Vector Machine":
    from sklearn.svm import SVR
    kernel = hyperparameter('kernel')
    if kernel == 'poly':
        regr = SVR(kernel=kernel, C=hyperparameter('C'), degree=hyperparameter('degree'))
    else:
        regr = SVR(kernel=kernel, C=hyperparameter('C'))
    model = regr.fit(dfxs, dfy)

write_fit(model)

set_metrics({
    'R2': model.score(dfxs, dfy),
})

A plot to detect model quality

In [ ]:
predicted = model.predict(dfxs)
residuals = predicted - dfy
from matplotlib import pyplot as plt
plt.figure(figsize=(6, 5))
plt.scatter(dfy, residuals)
plt.show()

Predicted vs observed in outcomes

In [ ]:
from matplotlib import pyplot as plt
outcome_field = model_cfg("outcome_field")
plt.figure(figsize=(6, 5))
plt.scatter(predicted, dfy)
plt.xlabel("Predicted "+outcome_field)
plt.ylabel("Observed "+outcome_field)
plt.show()

This is the predict code

In [ ]:
import pickle
import pandas

def predict(model_pickle_file, data_csv_file):
  with open(model_pickle_file, "rb") as input_file:
    mdl1 = pickle.load(input_file)
    predcsv = pandas.read_csv(data_csv_file)
    return {
        'yhat': list(mdl1.predict(predcsv))
      }